This was about building a folder full of images because it seemed like that's what pytorch is best at working with. This might still be a useful idea when I get to growing my dataset even further, but for now and it's specific form here it is a waste of space. I have now found a way to incorporate the image transformations into the dataset and it seems to work plenty fast enough now despite the fact that it seemed slow before. So even I do end up storing a bunch of small image files on disk I can do transformations in the model.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from PIL import Image
from skimage.measure import block_reduce
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

In [6]:
PATH = r'D:\Code\kural_work\movie_generation\data\\'
OUT_PATH = r'D:\Code\kural_work\movie_generation\data\IMAGES\\'
files = os.listdir(PATH)

In [3]:
nf_list = []
for file in files:
    if file[-4:] != '.tif': continue
    img_ = Image.open(PATH + file)
    nf_list.append(img_.n_frames)

In [6]:
nf = np.stack(nf_list)

In [8]:
nf = np.cumsum(nf)

In [40]:
nf

array([  400,   701,  1602,  2503,  3404,  4305,  5206,  5349,  5927,
        6228,  6697,  7724,  8358,  9259, 10160], dtype=int32)

In [75]:
print('test\n\
       a\n\
       b')

test
       a
       b


In [68]:
print('ab',end='\r')
print('cd\r')

cd


In [39]:
with open(PATH+'frame_list.pickle','wb') as f:
    pickle.dump(nf, f)

In [156]:
idx  = np.random.choice(range(nf[-1]*8*11))
# tmp,*_ = np.histogram(idx,bins=[0]+list(nf*11*8))
tmp_list = np.r_[0,nf*11*8]
tmp = np.logical_and(idx>tmp_list[:-1], idx<tmp_list[1:])
movie = np.nonzero(tmp)
print(idx,movie[0][0])

349531 5


In [142]:
tmp_list>100 & tmp_list<100

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [157]:
test_array = np.random.rand(15)

In [168]:
test_array.shape

(15,)

In [167]:
%%timeit
for _ in range(10000):
#     tmp = np.concatenate(([0],test_array))
    tmp = np.insert(test_array,0,0)

108 ms ± 523 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [80]:
np.asarray([True, False, False]) & np.asarray([True, False, True])

array([ True, False, False])

In [13]:
[0]+list(nf*11*8)

[0,
 35200,
 61688,
 140976,
 220264,
 299552,
 378840,
 458128,
 470712,
 521576,
 548064,
 589336,
 679712,
 735504,
 814792,
 894080]

I need to consider if it's even worth it to do the mirror and rotation. It probably is as long as I can deal with the data.

I also need to consider how to do the zero-to-one normalization<br>Answer: Just scale it for actually min (0) and actual max (2<sup>16</sup> - 1)

In [54]:
crop_width = 128
num_angles = 11
edge_param = 0.7
img = []
center_list = []
radius_list = []
mean_img_list = []
tmp_crop_list = []
min_max_list = []
for file in files:
    if file[-4:] != '.tif': continue
    print(file)
    img_ = Image.open(PATH + file)
    nf = img_.n_frames
    h = img_.height
    w = img_.width
    tmp_img = np.zeros((nf,h,w),dtype=np.float)
    for f in range(nf):
        img_.seek(f)
        tmp_img[f,:,:] = np.asarray(list(img_.getdata())).reshape(h,w)
    min_max_list.append((np.min(tmp_img), np.max(tmp_img)))
    mean_img = np.mean(tmp_img,axis=0)
    mean_img_list.append(mean_img)
    img_mask = (mean_img>(np.mean(mean_img)+np.std(mean_img))).astype(float)
    cy = sum(img_mask.T @ np.arange(h))/np.sum(img_mask)
    cx = sum(img_mask   @ np.arange(w))/np.sum(img_mask)
    center = (cy, cx)
    center_list.append(center)
    tmp = np.nonzero(img_mask)
    cell_height, cell_width = (max(tmp[i]) - min(tmp[i]) for i in range(2))
    cell_radius = np.mean([cell_height,cell_width])/2
    radius_list.append(cell_radius)
    r = cell_radius - edge_param*crop_width/2
    crop_cent = []
    for i in range(num_angles):
        crop_cent.append((np.int(center[0]+r*np.sin(2*np.pi*i/num_angles)), 
                          np.int(center[1]+r*np.cos(2*np.pi*i/num_angles))))
    for t, l in crop_cent:
        for mirror in iter([False,True]):
            for k in range(4):
                for f in range(nf):
                    tmp_crop = tmp_img[f,t-crop_width//2:t+crop_width//2,l-crop_width//2:l+crop_width//2]
                    if mirror: tmp_crop = tmp_crop.T
                    img.append(block_reduce(np.rot90(tmp_crop,k=k), (8,8), np.max))
    tmp_crop_list.append(tmp_crop)
print('Done')

3before_and_after.tif
as02_10m_2s_100ms_25p.tif
as03_30m_2s_50ms_100p.tif
as04_15m_1s_50ms_100p.tif
as05_15m_1s_50ms_100p.tif
as06_15m_1s_50ms_100p.tif
as07_15m_1s_50ms_100p.tif
s01_5m_2s_100ms_25p.tif
s02_39m_4s_50ms_100p.tif
s03_10m_2s_100ms_25p.tif
s04_32m_4s_50ms_100p.tif
s05_70m_4s_50ms_100p.tif
s06_42m_4s_50ms_100p.tif
s07_15m_1s_50ms_100p.tif
s08_30m_2s_50ms_100p.tif


In [11]:
crop_width = 128
num_angles = 11
edge_param = 0.7
img = []
center_list = []
radius_list = []
mean_img_list = []
min_max_list = []
out_file_counter = 1
for file in files:
    if file[-4:] != '.tif': continue
    print(file, end = ' ... ')
    img_ = Image.open(PATH + file)
    nf = img_.n_frames
    h = img_.height
    w = img_.width
    tmp_img = np.zeros((nf,h,w),dtype=np.float)
    for f in range(nf):
        img_.seek(f)
        tmp_img[f,:,:] = np.asarray(list(img_.getdata())).reshape(h,w)
    min_max_list.append((np.min(tmp_img), np.max(tmp_img)))
    mean_img = np.mean(tmp_img,axis=0)
    mean_img_list.append(mean_img)
    img_mask = (mean_img>(np.mean(mean_img)+np.std(mean_img))).astype(float)
    cy = sum(img_mask.T @ np.arange(h))/np.sum(img_mask)
    cx = sum(img_mask   @ np.arange(w))/np.sum(img_mask)
    center = (cy, cx)
    center_list.append(center)
    tmp = np.nonzero(img_mask)
    cell_height, cell_width = (max(tmp[i]) - min(tmp[i]) for i in range(2))
    cell_radius = np.mean([cell_height,cell_width])/2
    radius_list.append(cell_radius)
    r = cell_radius - edge_param*crop_width/2
    crop_cent = []
    for i in range(num_angles):
        crop_cent.append((np.int(center[0]+r*np.sin(2*np.pi*i/num_angles)), 
                          np.int(center[1]+r*np.cos(2*np.pi*i/num_angles))))
    for t, l in crop_cent:
        for mirror in iter([False,True]):
            for k in range(4):
                for f in range(nf):
                    img_.seek(f)
                    out = img_.crop((l-crop_width//2,
                                     t-crop_width//2,
                                     l+crop_width//2,
                                     t+crop_width//2))
                    if mirror: out = out.transpose(Image.FLIP_TOP_BOTTOM) #left_right doesn't work
                    for _ in range(k):
                        out = out.rotate(90) #rotate(180) doesn't work
                    out.save(OUT_PATH + f'{out_file_counter:08d}.tif')
                    out_file_counter += 1
    print('Done')

3before_and_after.tif ... Done
as02_10m_2s_100ms_25p.tif ... Done
as03_30m_2s_50ms_100p.tif ... Done
as04_15m_1s_50ms_100p.tif ... Done
as05_15m_1s_50ms_100p.tif ... Done
as06_15m_1s_50ms_100p.tif ... Done
as07_15m_1s_50ms_100p.tif ... Done
s01_5m_2s_100ms_25p.tif ... Done
s02_39m_4s_50ms_100p.tif ... Done
s03_10m_2s_100ms_25p.tif ... Done
s04_32m_4s_50ms_100p.tif ... Done
s05_70m_4s_50ms_100p.tif ... Done
s06_42m_4s_50ms_100p.tif ... Done
s07_15m_1s_50ms_100p.tif ... Done
s08_30m_2s_50ms_100p.tif ... Done


In [5]:
tmp = 234
print(OUT_PATH+f'{tmp:06d}.tif')

D:\Code\kural_work\movie_generation\data\IMAGES\\000234.tif


In [55]:
img = np.stack(img)

In [57]:
img = img/(2**16-1)

In [58]:
with open(PATH + 'img_data.pickle', 'wb') as f:
    pickle.dump(img, f)

In [ ]:
with open(PATH + 'img_data.pickle', 'rb') as f:
    img = pickle.load(f)